<table style="float:left; border:none">
   <tr style="border:none; background-color: #ffffff">
       <td style="border:none">
           <a href="http://bokeh.pydata.org/">     
           <img 
               src="assets/bokeh-transparent.png" 
               style="width:50px"
           >
           </a>    
       </td>
       <td style="border:none">
           <h1>Bokeh 教程</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2>06. 关联和交互</h2></div>

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

在上一章，我们知道了怎么把多个图表（plot）放在一个布局（layout）里。现在，我们就来看看怎么把不同的图表联系起来，怎么把图表和控件（widget）联系起来。

# Linked Interactions

不同的Bokeh图表之间是可以有联系的。例如，两个（或更多）图的范围可能是由联系的，当一个图被平移（或放大，或其范围变化），其它的图也相应更新保持一致。还可以将两个图的选择（selection）联系起来，当在一个图上选择某项时，另一个图中对应的项也被选中。

## Linked panning（相连的平移）

相连的平移（多个图的range保持同步）在Bokeh里非常容易。您只需让两个（或多个）图共享适当的range对象就可以了。下面的例子展示了如何通过三种不同的方式连接这些图的range：

In [2]:
from bokeh.layouts import gridplot

x = list(range(11))
y0, y1, y2 = x, [10-i for i in x], [abs(i-5) for i in x]

plot_options = dict(width=250, plot_height=250, tools='pan,wheel_zoom')

# create a new plot
s1 = figure(**plot_options)
s1.circle(x, y0, size=10, color="navy")

# create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range, **plot_options)
s2.triangle(x, y1, size=10, color="firebrick")

# create a new plot and share only one range
s3 = figure(x_range=s1.x_range, **plot_options)
s3.square(x, y2, size=10, color="olive")

p = gridplot([[s1, s2, s3]])

# show the results
show(p)

In [3]:
# EXERCISE: create two plots in a gridplot, and link their ranges


## Linked brushing

相连的选择（selection）通过类似的方式完成，在图块之间共享数据源。注意，通常 ``bokeh.plotting`` 和 ``bokeh.charts`` 会自动创建一个缺省数据源。然而，为了共享数据源，我们必须手动创建它们并显式地传递。下面的例子说明了这一点：

In [4]:
from bokeh.models import ColumnDataSource

x = list(range(-20, 21))
y0, y1 = [abs(xx) for xx in x], [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_select,lasso_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=300, height=300)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

In [5]:
# EXERCISE: create two plots in a gridplot, and link their data sources



# Hover Tools（悬停工具）

Bokeh有一个悬停的工具，当用户将鼠标悬停在一个特定的标记符号（glyph）上时，可以在一个弹出框里显示更多的信息。基本的悬停工具配置相当于提供一个 ``(name, format)`` 元组列表。完整的细节请参考用户指南 [hovertool](http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#hovertool)。

下面的例子展示了悬停工的一些基本使用，悬停信息在utils.py中定义：

In [6]:
from bokeh.models import HoverTool

source = ColumnDataSource(
        data=dict(
            x=[1, 2, 3, 4, 5],
            y=[2, 5, 8, 2, 7],
            desc=['A', 'b', 'C', 'd', 'E'],
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("desc", "@desc"),
        ]
    )

p = figure(plot_width=300, plot_height=300, tools=[hover], title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)

show(p)

# Widgets（控件）

Bokeh直接集成了一个小的基本控件集。这些控件和Bokeh服务器或 ``CustomJS`` 模型协作使用可以加入更多的互动的功能。您可以在用户指南的 [Adding Widgets](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html#adding-widgets) 部分看到完整的widget列表和示例代码。

要使用小部件，就像您将使用一个绘图对象一样，将它们包括在布局中：

In [7]:
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider


slider = Slider(start=0, end=10, value=1, step=.1, title="foo")

show(widgetbox(slider))

In [8]:
# EXERCISE: create and show a Select widget 


# CustomJS Callbacks（CustomJS回调）

In [13]:
from bokeh.models import TapTool, CustomJS, ColumnDataSource

callback = CustomJS(code="alert('hello world')")
tap = TapTool(callback=callback)

p = figure(plot_width=600, plot_height=300, tools=[tap])

p.circle(x=[1, 2, 3, 4, 5], y=[2, 5, 8, 2, 7], size=20)

show(p)

## 很多地方可以添加回调

* Widgets - Button, Toggle, Dropdown, TextInput, AutocompleteInput, Select, Multiselect, Slider, (DateRangeSlider), DatePicker,
* Tools - TapTool, BoxSelectTool, HoverTool,
* Selection - ColumnDataSource, AjaxDataSource, BlazeDataSource, ServerDataSource
* Ranges - Range1d, DataRange1d, FactorRange


## Callbacks for widgets（控件的回调）

具有关联值的控件可以附上小的JavaScript动作。这些动作（也被称为“回调”）在控件的值改变时被执行。为了更容易从JavaScript引用特定的Bokeh模型（例如，数据源或标记符），``CustomJS`` 对象接受一个dictionary类型的“参数”，映射名字到Bokeh模型。对应的JavaScript模型自动提供给 ``CustomJS`` 代码。

下面的例子展示了一个附加到滑块的动作，滑块每次移动时都会更新数据源：

In [10]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = [x*0.005 for x in range(0, 201)]

source = ColumnDataSource(data=dict(x=x, y=x))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.get('data');
    var f = slider.value;
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")
slider.js_on_change('value', update_curve)


show(column(slider, plot))

## Calbacks for selections（选择的回调）

也可以在用户选择（例如，框、点、套索）更改时执行JavaScript动作。这是通过将同样的customjs对象附加到要选择的数据源实现的。

下面的示例稍稍复杂，展示通过响应一个标记符的选择来更新另一个标记符的数据源：

In [12]:
from random import random

x = [random() for x in range(500)]
y = [random() for y in range(500)]
color = ["navy"] * len(x)

s = ColumnDataSource(data=dict(x=x, y=y, color=color))
p = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p.circle('x', 'y', color='color', size=8, source=s, alpha=0.4)

s2 = ColumnDataSource(data=dict(xm=[0,1],ym=[0.5, 0.5]))
p.line(x='xm', y='ym', color="orange", line_width=5, alpha=0.6, source=s2)

s.callback = CustomJS(args=dict(s2=s2), code="""
    var inds = cb_obj.get('selected')['1d'].indices;
    var d = cb_obj.get('data');
    var ym = 0
    
    if (inds.length == 0) { return; }
    
    for (i = 0; i < d['color'].length; i++) {
        d['color'][i] = "navy"
    }
    for (i = 0; i < inds.length; i++) {
        d['color'][inds[i]] = "firebrick"
        ym += d['y'][inds[i]]
    }
    
    ym /= inds.length
    s2.get('data')['ym'] = [ym, ym]
    
    cb_obj.trigger('change');
    s2.trigger('change');
""")

show(p)

# 更多
更多交互，参考用户指南 - http://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html